In [2]:
import pymongo

In [3]:
client = pymongo.MongoClient()
db = client.metrics
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'metrics')

## Iterate Over All of the Collections in the DB

In [4]:
for col in sorted(db.collection_names()):
    print col

CB_TAGGING_TD_AVG_PERCEPTRON_MULTICLASS
CB_TAGGING_TD_CRF
CB_TAGGING_TD_CRF_LBL_POWERSET
CB_TAGGING_TD_CRF_LBL_POWERSET_HYPERPARAM_OPT
CB_TAGGING_TD_CRF_MOST_COMMON_TAG
CB_TAGGING_TD_FEAT_SELECTION
CB_TAGGING_TD_HMM
CB_TAGGING_TD_HMM_LBL_POWERSET
CB_TAGGING_TD_WINDOW_CLASSIFIER_BR
CB_TAGGING_TD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING
CB_TAGGING_TD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
CB_TAGGING_TD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING
CB_TAGGING_VD_AVG_PERCEPTRON_MULTICLASS
CB_TAGGING_VD_CRF
CB_TAGGING_VD_CRF_LBL_POWERSET
CB_TAGGING_VD_CRF_LBL_POWERSET_HYPERPARAM_OPT
CB_TAGGING_VD_CRF_MOST_COMMON_TAG
CB_TAGGING_VD_FEAT_SELECTION
CB_TAGGING_VD_HMM
CB_TAGGING_VD_HMM_LBL_POWERSET
CB_TAGGING_VD_WINDOW_CLASSIFIER_BR
CB_TAGGING_VD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING
CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING
SC_TAGGING_TD_AVG_PERCEPTRON_MULTICLASS
SC_TAGGING_TD_CRF
SC_TAGGI

## Clean Data

In [167]:
for col in db.collection_names():
    collection = db[col]
    
    drop = False
    if "BrillTagger" in col or "FASTTEXT" in col or "CAUSE_EFFECT" in col:
        drop = True
        
    #if col[-3:] in set("_TD,_VD".split(",")):
    #    drop = True
        
    if not drop:
        if "TAGGING" in col:
            missing_count = collection.find({"MICRO_F1": {'$exists': False}}).count()
            if missing_count > 0:
                print col + " - Deleting " + str(missing_count) + " rows without micro F1 scores"
                result = collection.delete_many({"MICRO_F1": {'$exists': False}})
                print str(result.deleted_count) + " successfully deleted"
        else:
            prefix = col[0:2]
            # Don't drop system tables (like the indexes collection)
            if prefix == "SC" or prefix == "CB":
                drop = True
    final_count = collection.find({}).count()
    if final_count == 0:
        drop = True
        print "Empty collection - " + col
    
    if drop:
        print "Dropping " + col + " ..."
        collection.drop()

Dropping CB_SENT_TD ...
Dropping CB_SENT_TD_CAUSE_EFFECT_LBLS ...
Dropping CB_SENT_VD ...
Dropping CB_SENT_VD_CAUSE_EFFECT_LBLS ...
CB_TAGGING_TD - Deleting 58 rows without micro F1 scores
58 successfully deleted
Empty collection - CB_TAGGING_TD
Dropping CB_TAGGING_TD ...
Dropping CB_TAGGING_TD_BrillTagger_HMM ...
Dropping CB_TAGGING_TD_CAUSE_EFFECT_LBLS ...
CB_TAGGING_TD_CRF - Deleting 5 rows without micro F1 scores
5 successfully deleted
Dropping CB_TAGGING_TD_FASTTEXT_LBL_POWERSET ...
CB_TAGGING_TD_FEAT_SELECTION - Deleting 643 rows without micro F1 scores
643 successfully deleted
CB_TAGGING_VD - Deleting 58 rows without micro F1 scores
58 successfully deleted
Empty collection - CB_TAGGING_VD
Dropping CB_TAGGING_VD ...
Dropping CB_TAGGING_VD_BrillTagger_HMM ...
Dropping CB_TAGGING_VD_CAUSE_EFFECT_LBLS ...
CB_TAGGING_VD_CRF - Deleting 3 rows without micro F1 scores
3 successfully deleted
Dropping CB_TAGGING_VD_FASTTEXT_LBL_POWERSET ...
CB_TAGGING_VD_FEAT_SELECTION - Deleting 643 rows

# Clean Bad Data (Skin Cancer in CB Data and Vice Versa

**Need to validate we computed all of the range of values for each feature set, and didn't run a long experiment and dumped the data in the wrong location. Looks like not**

In [33]:
dates = db["CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING"].find({"parameters.folder": 
    {"$regex": ".*Coral.*"}}, {"asof": 1, "_id": 0})
s_dates = sorted(map(lambda r: str(r["asof"]),list(dates)))
s_dates[0], s_dates[-1], len(s_dates)

('2016-11-05 16:01:53.798000', '2016-11-06 16:01:15.702000', 119)

In [30]:
dates = db["SC_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING"].find({}, {"asof": 1, "_id": 0})
s_dates = sorted(map(lambda r: str(r["asof"]),list(dates)))
s_dates[0], s_dates[-1], len(s_dates)

('2016-11-06 09:03:46.387000', '2016-11-06 16:15:00.545000', 60)

In [38]:
for col in db.collection_names():
    collection = db[col]
    if col.startswith("CB_"):        
        count = collection.find({"parameters.folder": 
                {"$regex": ".*SkinCancer.*"}}).count()
        if count > 0:
            print col, "bad records successfully deleted"
    if col.startswith("SC_"):
        count = collection.find({"parameters.folder": 
                {"$regex": ".*Coral.*"}}).count()
        if count > 0:
            print col, "bad records successfully deleted"

In [37]:
for col in db.collection_names():
    collection = db[col]
    if col.startswith("CB_"):        
        del_rslt = collection.delete_many({"parameters.folder": 
                {"$regex": ".*SkinCancer.*"}})
        if del_rslt.deleted_count > 0:
            print col, str(del_rslt.deleted_count) + " successfully deleted"
    if col.startswith("SC_"):
        del_rslt = collection.delete_many({"parameters.folder": 
                {"$regex": ".*Coral.*"}})
        if del_rslt.deleted_count > 0:
            print col, str(del_rslt.deleted_count) + " successfully deleted"

# *NOTE* - I CREATED A BACKUP in GOOGLE DRIVE, so all data can be recovered (but needs cleaning once more)

## Get Collection Update Dates

In [5]:
latest_updates = dict()
oldest_updates = dict()

for col in db.collection_names():
    collection = db[col]
    if "TAGGING" in col:
        newest_row = collection.find({}).sort("asof", pymongo.DESCENDING).next()
        oldest_row = collection.find({}).sort("asof", pymongo.ASCENDING).next()
        latest_updates[col] = newest_row["asof"].date()
        oldest_updates[col] = oldest_row["asof"].date()

### What Model Types and Variants were Hyper Parameter Optimized?

In [6]:
sorted([k.replace("CB_TAGGING_VD_", "") for k in latest_updates.keys() if "VD" in k and "CB" in k])

[u'AVG_PERCEPTRON_MULTICLASS',
 u'CRF',
 u'CRF_LBL_POWERSET',
 u'CRF_LBL_POWERSET_HYPERPARAM_OPT',
 u'CRF_MOST_COMMON_TAG',
 u'FEAT_SELECTION',
 u'HMM',
 u'HMM_LBL_POWERSET',
 u'WINDOW_CLASSIFIER_BR',
 u'WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING',
 u'WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS',
 u'WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING']

In [7]:
import datetime

def sort_key(tpl):
    col, dt = tpl
    return dt

by_date = sorted(oldest_updates.items(), key = lambda tpl: sort_key(tpl))[::-1]
for col, dt in by_date:
    print col.ljust(80), str(dt)

SC_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING       2016-11-06
SC_TAGGING_VD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING                            2016-11-06
SC_TAGGING_TD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING       2016-11-06
SC_TAGGING_TD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING                            2016-11-06
CB_TAGGING_TD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING       2016-11-05
CB_TAGGING_VD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING                            2016-11-05
CB_TAGGING_TD_WINDOW_CLASSIFIER_BR_HYPER_PARAM_TUNING                            2016-11-05
CB_TAGGING_VD_WINDOW_CLASSIFIER_BR                                               2016-11-05
CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS_HYPER_PARAM_TUNING       2016-11-05
CB_TAGGING_TD_WINDOW_CLASSIFIER_BR                                               2016-11-05
SC_TAGGING_TD_HMM_LBL_POWERSET                                                  